<a href="https://colab.research.google.com/github/wz30/duke_edge_computing/blob/main/dataset_pre_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# test other dataset here
# MNIST 60,000 10,000 10 28 × 28 × 1 hand-written digits
# SVHN 73,257 26,032 10 32 × 32 × 3 cropped digits from street view
# FEMNIST 71,090 8,085 62 28 × 28 × 1 hand-written characters (federated learning setting)
# Chars74K 58,097 17,398 62 28 × 28 × 1 natural images, hand-written and font characters
# FASHION 60,000 10,000 10 28 × 28 × 1 fashion items
# CIFAR-10 50,000 10,000 10 32 × 32 × 3 various object classes
# CIFAR-100 50,000 10,000 100 32 × 32 × 3 various object classes
!pip install torch==1.4.0

     |████████████████████████████████| 753.4MB 23kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.4.0 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101


In [ ]:
import torchvision
import torchvision.transforms as transforms
import torch
import matplotlib.pyplot as plt
import numpy as np
plt.figure(figsize = (3,3)) #define the image size
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
print(torch.__version__)

1.8.0+cu101


<Figure size 216x216 with 0 Axes>

In [ ]:
#transforming the PIL Image to tensors
trainset = torchvision.datasets.FashionMNIST(root = "./data", train = True, download = True, transform = transforms.ToTensor())
testset = torchvision.datasets.FashionMNIST(root = "./data", train = False, download = True, transform = transforms.ToTensor())

In [ ]:
!wget www.di.ens.fr/~lelarge/MNIST.tar.gz
!tar -zxvf MNIST.tar.gz
#MNIST
minist_train = torchvision.datasets.MNIST('./', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
minist_test = torchvision.datasets.MNIST('./', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

--2021-03-19 03:41:31--  http://www.di.ens.fr/~lelarge/MNIST.tar.gz
Resolving www.di.ens.fr (www.di.ens.fr)... 129.199.99.14
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.di.ens.fr/~lelarge/MNIST.tar.gz [following]
--2021-03-19 03:41:32--  https://www.di.ens.fr/~lelarge/MNIST.tar.gz
Connecting to www.di.ens.fr (www.di.ens.fr)|129.199.99.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/x-gzip]
Saving to: ‘MNIST.tar.gz’

MNIST.tar.gz            [            <=>     ]  33.20M  11.8MB/s    in 2.8s    

2021-03-19 03:41:35 (11.8 MB/s) - ‘MNIST.tar.gz’ saved [34813078]

MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/tra

In [ ]:
# minist_train  trainset
trainset_list = []
for i in trainset:
  trainset_list.append(i)
print(len(trainset_list))
for i in minist_train:
  trainset_list.append(i)
print(len(trainset_list))
# testset_list = []
# for i in testset:
#   testset_list.append(i)
# for i in minist_test:
#   testset_list.append(i)
# print(len(testset_list))

# merge_train # 120k

60000
120000


In [ ]:
#loading the training data from trainset
trainloader = torch.utils.data.DataLoader(trainset_list[45000:60500], batch_size=4, shuffle = True)
# trainloader = torch.utils.data.DataLoader(, batch_size=4, shuffle = True)
#loading the test data from testset
print(type(minist_test))
testloader = torch.utils.data.DataLoader(minist_test, batch_size=4, shuffle=False)

<class 'torchvision.datasets.mnist.MNIST'>


In [ ]:
classes = ('T-Shirt','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle Boot')
def imshow(img):
    npimg = img.numpy() #convert the tensor to numpy for displaying the image
    #for displaying the image, shape of the image should be height * width * channels 
    plt.imshow(np.transpose(npimg, (1, 2, 0))) 
    plt.show()

In [ ]:
# class Net(nn.Module):
#   def init(self):
#     super(Net, self).init()
#     self.cnn_model = nn.Sequential(
#         nn.Conv2d(3, 6, kernel_size = 5), #(N, 1, 28, 28) -> (N, 6, 24, 24)
#         nn.Tanh(),
#         nn.AvgPool2d(2, stride = 2), #(N, 6, 24, 24) -> (N, 6, 12, 12)
#         nn.Conv2d(6, 16, kernel_size = 5), #(N, 6, 12, 12) -> (N, 6, 8, 8)
#         nn.Tanh(),
#         nn.AvgPool2d(2, stride = 2)) #(N, 6, 8, 8) -> (N, 16, 4, 4)
#     self.fc_model = nn.Sequential(
#         nn.Linear(256, 120), # (N, 256) -> (N, 120)
#         nn.Tanh(),
#         nn.Linear(120, 84), # (N, 120) -> (N, 84)
#         nn.Tanh(),
#         nn.Linear(84, 10))  # (N, 84)  -> (N, 10)) #10 classes
#   def forward(self, x):
#     x = self.cnn_model(x)     
#     x = x.view(x.size(0), -1)     
#     x = self.fc_model(x)     
#     return x

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False

args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# device = torch.device("cpu")
device
# print(torch.cuda.device_count())
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)

device(type='cuda')

In [ ]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        # model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        # model.get() # <-- NEW: get the model back
        if batch_idx % args.log_interval == 0:
            # loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [ ]:
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) # TODO momentum is not supported at the moment

for epoch in range(1, args.epochs + 1):
    train(args, model, device, trainloader, optimizer, epoch)
    test(args, model, device, testloader)

if (args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/248000 (0%)]	Loss: 2.307578
Train Epoch: 1 [1920/248000 (1%)]	Loss: 2.262147
Train Epoch: 1 [3840/248000 (2%)]	Loss: 2.346854
Train Epoch: 1 [5760/248000 (2%)]	Loss: 2.234870
Train Epoch: 1 [7680/248000 (3%)]	Loss: 2.269196
Train Epoch: 1 [9600/248000 (4%)]	Loss: 2.052919
Train Epoch: 1 [11520/248000 (5%)]	Loss: 2.114006
Train Epoch: 1 [13440/248000 (5%)]	Loss: 1.930754
Train Epoch: 1 [15360/248000 (6%)]	Loss: 1.840923
Train Epoch: 1 [17280/248000 (7%)]	Loss: 1.579124
Train Epoch: 1 [19200/248000 (8%)]	Loss: 1.067855
Train Epoch: 1 [21120/248000 (9%)]	Loss: 2.276411
Train Epoch: 1 [23040/248000 (9%)]	Loss: 1.682679
Train Epoch: 1 [24960/248000 (10%)]	Loss: 0.769681
Train Epoch: 1 [26880/248000 (11%)]	Loss: 1.463723
Train Epoch: 1 [28800/248000 (12%)]	Loss: 1.911585
Train Epoch: 1 [30720/248000 (12%)]	Loss: 0.749596
Train Epoch: 1 [32640/248000 (13%)]	Loss: 1.579467
Train Epoch: 1 [34560/248000 (14%)]	Loss: 1.364793
Train Epoch: 1 [36480/248000 (15%)]	Loss: 1.286079
Tr